<a href="https://colab.research.google.com/github/sreejamlnlp/Python/blob/main/PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!pip install requests==2.32.3

In [52]:
!pip install -U langfuse urllib3

In [53]:
!pip install crewai crewai-tools langchain_community openlit transformers

In [54]:
!pip install openinference-instrumentation-crewai openinference-instrumentation-litellm

In [55]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [56]:
from crewai import Agent, Task, Crew, Process

In [57]:
import base64
import os

from google.colab import userdata
from crewai import LLM

In [58]:
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [100]:
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.3, #0.7 --> Uncertainities will go high, that means the LLM will act as a creative system.
)

In [80]:
#llm = LLM(
#    model="openai/gpt-4o-mini", # call model by provider/model_name
#    temperature=0.8,
#    max_tokens=150,
#    top_p=0.9,
#    frequency_penalty=0.1,
#    presence_penalty=0.1,
#    seed=42
#)'

In [101]:
#Database - Supabase- S3- DB-Documents
# ==============================================================================
# 3. Mock Data for Market Research
# ==============================================================================
# This pre-defined data simulates user feedback for the IndieFoodie app.
mock_user_feedback = """
- Snippet 1 (from Reddit r/cooking): "I love using IndieFoodie to 'shop by intention' for a specific dinner like 'Taco Night', but I wish it could help me plan my entire week. I'm busy and want to just say 'plan a week of healthy, low-effort meals' and get a full shopping list."
- Snippet 2 (from Twitter): "@IndieFoodie your 'Italian Feast' intention was a hit! But my partner has a gluten allergy, and it was hard to find suitable recipe swaps. Can you please add dietary filters (vegan, gluten-free, keto) to your intentions?"
- Snippet 3 (from a Food Tech blog): "While IndieFoodie's 'Shop by Intention' is revolutionary, it lacks scalability for events. A user can't easily adjust an 'intention' for a dinner party of 12, a major feature in competitor apps like Instacart that are now testing their own recipe-to-cart features."
- Snippet 4 (from Play Store Review): "Great app, 5 stars for the concept. But please add a feature to create my OWN intentions. I have a set of family recipes I make all the time, and I'd love to save them as a reusable shopping intention."
- Snippet 5 (from Twitter): "Just tried to use @IndieFoodie for my weekly meal prep. The lack of a multi-meal or weekly planning feature is a dealbreaker. Going back to manual lists for now."
"""

In [102]:
# Serper API Configuration
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

In [103]:
from crewai_tools import SerperDevTool # <--- Import SerperDevTool

In [104]:
search_tool = SerperDevTool()

In [105]:
# ==============================================================================
# 4. Agent Definitions
# ==============================================================================
# Agent 1: Initial Feedback Analyzer
initial_feedback_analyzer = Agent(
    role='Market Research Analyst',
    goal='Analyze provided user feedback to identify key pain points and feature requests for the IndieFoodie app.',
    backstory="An expert in analyzing qualitative data to find actionable insights.",
    verbose=True,
    llm = llm
)

# Agent 2: The Research Director
research_director = Agent(
    role='Director of Market Research',
    goal='Conduct a deep-dive web search based on initial findings and a specific research topic.',
    backstory="A master researcher who uses online sources to validate hypotheses and uncover detailed market intelligence.",
    tools=[search_tool],
    verbose=True,
    llm = llm
)

In [106]:
# Agent 3: The Feature Ideator
feature_ideator = Agent(
    role='Principal Product Strategist',
    goal='Synthesize all research findings into a single, high-impact feature proposal for the IndieFoodie app.',
    backstory="A visionary product leader who translates raw data into brilliant, concrete feature concepts.",
    verbose=True,
    llm = llm
)

In [107]:
# Agent 4: The Internal Pitch Writer
pitch_writer = Agent(
    role='Executive Communications Lead',
    goal='Draft a compelling 60-second executive pitch for a new feature idea.',
    backstory="A master of creating concise, persuasive narratives for busy executives.",
    verbose=True,
    llm = llm
)

In [108]:
# Agent 5: The PRD Drafter
prd_drafter = Agent(
    role='Senior Technical Product Manager',
    goal='Create a clear, one-page Product Requirements Document (PRD) for the proposed feature.',
    backstory="A meticulous planner who bridges concepts and actionable engineering plans.",
    verbose=True,
    llm = llm
)

In [109]:
# Agent 6: Asana Task Formatter
task_formatter = Agent(
    role='Agile Task Formatting Specialist',
    goal='Format the key user stories from a PRD into a structured JSON array for Asana.',
    backstory="An efficiency expert who converts requirements into clean, structured JSON.",
    verbose=True,
    llm = llm
)

In [110]:
# ==============================================================================
# 6. Task Definitions (6 Total Tasks)
# ==============================================================================
# Task 1: Initial Feedback Analysis
initial_analysis_task = Task(
    description=(
        f"Analyze the following user feedback about the 'IndieFoodie' app. Summarize the top 3-4 recurring themes, pain points, or feature requests.\n\n DATA:\n {mock_user_feedback}"
    ),
    expected_output="A concise summary of the top 3-4 user pain points and feature requests.",
    agent=initial_feedback_analyzer
)

In [111]:
# Task 2: Deep-Dive Market Research
research_task = Task(
    description=(
        "A Product Manager has reviewed the initial feedback summary and wants a deeper investigation. "
        "Conduct a targeted web search on the following topic: '{research_topic}'. "
        "Find information on how competitor apps are implementing similar features, what technologies they are using, "
        "and overall user sentiment towards those features. Synthesize your findings into a research report."
    ),
    expected_output="A detailed research report summarizing competitor features, market trends, and user expectations for the given research topic.",
    agent=research_director,
    context=[initial_analysis_task]
)

In [112]:
# Task 3: Feature Ideation
ideation_task = Task(
    description=(
        "Analyze the initial feedback summary and the deep-dive research report. "
        "Synthesize all insights to propose a single, high-impact feature for the IndieFoodie app. "
        "Your output must be a JSON object with keys 'user_problem', 'proposed_feature_name', and 'feature_concept'."
    ),
    expected_output=(
        "A JSON object containing a concise definition of the core user problem, a creative name for the proposed feature, "
        "and a one-paragraph description of the feature concept."
    ),
    agent=feature_ideator,
    context=[initial_analysis_task, research_task]
)

In [113]:
# Task 4: 60-Second Pitch
pitch_task = Task(
    description=(
        "Using the feature proposal, write a compelling 60-second executive pitch for the IndieFoodie leadership team."
    ),
    expected_output="A concise, persuasive paragraph that effectively sells the new feature idea.",
    agent=pitch_writer,
    context=[ideation_task]
)

In [114]:
# Task 5: PRD Creation
prd_task = Task(
    description=(
        "Take the feature concept and executive pitch and draft a comprehensive, one-page Product Requirements Document (PRD). "
        "The PRD must be in Markdown format and include these sections: Feature Name, Problem Statement, Solution/Concept, "
        "Key User Stories (at least 3), Success Metrics, and Out of Scope."
    ),
    expected_output="A well-structured, one-page PRD in Markdown format.",
    agent=prd_drafter,
    context=[ideation_task, pitch_task]
)

In [115]:
# Task 6: Format Tasks for Asana
task_formatting_task = Task(
    description=(
        "Parse the 'Key User Stories' section from the final PRD. For each user story, create a JSON object that represents a task. "
        "Each JSON object must have two keys: 'task_name' and 'task_notes'. The 'task_name' should be the user story itself, "
        "and the 'task_notes' should be a brief elaboration. Your final output should be a single JSON array containing these task objects."
    ),
    expected_output=(
        "A JSON array of task objects, where each object has 'task_name' and 'task_notes' keys."
    ),
    agent=task_formatter,
    context=[prd_task]
)

In [116]:
# ==============================================================================
# 7. Crew Instantiation and Kickoff
# ==============================================================================
product_innovation_crew = Crew(
    agents=[
        initial_feedback_analyzer,
        research_director,
        feature_ideator,
        pitch_writer,
        prd_drafter,
        task_formatter
    ],
    tasks=[
        initial_analysis_task,
        research_task,
        ideation_task,
        pitch_task,
        prd_task,
        task_formatting_task
    ],
    process=Process.sequential,
    verbose=True
)

In [117]:
# Define the inputs for the kickoff
crew_inputs = {
    'user_feedback': mock_user_feedback,
    'research_topic': 'Community and social sharing features in recipe and grocery apps'
}

# Kick off the crew's work
print("🚀 Kicking off the AI Product Innovation Crew for IndieFoodie...")
result = product_innovation_crew.kickoff(inputs=crew_inputs)

🚀 Kicking off the AI Product Innovation Crew for IndieFoodie...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 28b78a3f-7f4b-4ca9-a690-124afecec47d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Analyze the following user feedback about the 'IndieFoodie' app. Summarize the top 3-4 recurring         │
│  themes, pain points, or feature requests.                                                                      │
│                                                                                                                 │
│   DATA:                                                                                                         │
│                                                                                                                 │
│  - Snippet 1 (from Reddit r/cooking): "I love using IndieFoodie to 'shop by intention' for a specific dinner    │
│  like 'Taco Night', but I wish it could help me plan my entire week. I'm busy and want to just say 'plan a      │
│  week of healthy, low-effort meals' and get a full shopping list."                                              │
│  - Snippet 2 (from Twitter): "@IndieFoodie your 'Italian Feast' intention was a hit! But my partner has a       │
│  gluten allergy, and it was hard to find suitable recipe swaps. Can you please add dietary filters (vegan,      │
│  gluten-free, keto) to your intentions?"                                                                        │
│  - Snippet 3 (from a Food Tech blog): "While IndieFoodie's 'Shop by Intention' is revolutionary, it lacks       │
│  scalability for events. A user can't easily adjust an 'intention' for a dinner party of 12, a major feature    │
│  in competitor apps like Instacart that are now testing their own recipe-to-cart features."                     │
│  - Snippet 4 (from Play Store Review): "Great app, 5 stars for the concept. But please add a feature to create  │
│  my OWN intentions. I have a set of family recipes I make all the time, and I'd love to save them as a          │
│  reusable shopping intention."                                                                                  │
│  - Snippet 5 (from Twitter): "Just tried to use @IndieFoodie for my weekly meal prep. The lack of a multi-meal  │
│  or weekly planning feature is a dealbreaker. Going back to manual lists for now."                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The top 4 user pain points and feature requests for the IndieFoodie app are:                                   │
│                                                                                                                 │
│  1.  **Lack of Weekly Meal Planning/Multi-Meal Planning:** Users repeatedly express the need for a feature      │
│  that allows them to plan meals for an entire week or multiple meals at once. They want to input dietary        │
│  preferences and receive a comprehensive shopping list, streamlining their grocery shopping experience. This    │
│  includes the ability to specify parameters like "healthy, low-effort meals."                                   │
│                                                                                                                 │
│  2.  **Insufficient Dietary Filters:** Users desire more robust dietary filters within the "Shop by Intention"  │
│  feature. The current filtering system is inadequate, particularly for users with specific allergies or         │
│  dietary restrictions (e.g., gluten-free, vegan, keto). They need to easily identify and swap recipes to        │
│  accommodate these needs.                                                                                       │
│                                                                                                                 │
│  3.  **Inability to Customize/Create Intentions:** Users want the ability to create their own custom            │
│  "intentions" based on their personal recipes and preferences. This would allow them to save and reuse their    │
│  favorite family meals as reusable shopping intentions, enhancing personalization and convenience.              │
│                                                                                                                 │
│  4.  **Limited Scalability for Events/Large Groups:** The app lacks the functionality to easily adjust          │
│  existing "intentions" for larger groups or events, such as dinner parties. Users find it difficult to scale    │
│  recipes and shopping lists for a greater number of people, a feature present in competitor apps.               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Agile Task Formatting Specialist                                                                        │
│                                                                                                                 │
│  Task: Parse the 'Key User Stories' section from the final PRD. For each user story, create a JSON object that  │
│  represents a task. Each JSON object must have two keys: 'task_name' and 'task_notes'. The 'task_name' should   │
│  be the user story itself, and the 'task_notes' should be a brief elaboration. Your final output should be a    │
│  single JSON array containing these task objects.                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Agile Task Formatting Specialist                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```json                                                                                                        │
│  [                                                                                                              │
│    {                                                                                                            │
│      "task_name": "As a user with a gluten allergy, I want to be able to easily filter recipes and ingredients  │
│  to exclude gluten-containing items, so I can confidently create meal plans that meet my dietary needs.",       │
│      "task_notes": "Implement filtering functionality for recipes and ingredients based on gluten content.      │
│  This allows users with gluten allergies to easily identify and exclude items that are not suitable for them."  │
│    },                                                                                                           │
│    {                                                                                                            │
│      "task_name": "As a user who enjoys cooking my own recipes, I want to be able to add my own recipes to the  │
│  \"Shop by Intention\" feature and generate shopping lists automatically, so I can easily incorporate my        │
│  favorite meals into my weekly plan.",                                                                          │
│      "task_notes": "Enable users to add their own recipes to the 'Shop by Intention' feature. The system        │
│  should automatically generate shopping lists based on the ingredients in the user-added recipes."              │
│    },                                                                                                           │
│    {                                                                                                            │
│      "task_name": "As a user who frequently cooks for different group sizes, I want to be able to easily scale  │
│  recipes and shopping lists for different numbers of people, so I can avoid food waste and ensure I have        │
│  enough ingredients for everyone.",                                                                             │
│      "task_notes": "Develop a recipe scaling tool that allows users to adjust recipe quantities based on the    │
│  number of people they are cooking for. The tool should also automatically update the shopping list to reflect  │
│  the scaled quantities."                                                                                        │
│    },                                                                                                           │
│    {                                                                                                            │
│      "task_name": "As a busy parent, I want to quickly create a meal plan for the week that caters to my        │
│  family's dietary needs and preferences, so I can save time and reduce stress.",                                │
│      "task_notes": "Provide a streamlined workflow for creating weekly meal plans that takes into account       │
│  dietary needs and preferences. The goal is to make the meal planning process as quick and easy as possible     │
│  for busy parents."                                                                                             │
│    }                                                   

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0de97a18-f3c3-464d-9398-b181fb8c9db1                                                                     │
│  Agent: Agile Task Formatting Specialist                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 28b78a3f-7f4b-4ca9-a690-124afecec47d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```json                                                                                          │
│  [                                                                                                              │
│    {                                                                                                            │
│      "task_name": "As a user with a gluten allergy, I want to be able to easily filter recipes and ingredients  │
│  to exclude gluten-containing items, so I can confidently create meal plans that meet my dietary needs.",       │
│      "task_notes": "Implement filtering functionality for recipes and ingredients based on gluten content.      │
│  This allows users with gluten allergies to easily identify and exclude items that are not suitable for them."  │
│    },                                                                                                           │
│    {                                                                                                            │
│      "task_name": "As a user who enjoys cooking my own recipes, I want to be able to add my own recipes to the  │
│  \"Shop by Intention\" feature and generate shopping lists automatically, so I can easily incorporate my        │
│  favorite meals into my weekly plan.",                                                                          │
│      "task_notes": "Enable users to add their own recipes to the 'Shop by Intention' feature. The system        │
│  should automatically generate shopping lists based on the ingredients in the user-added recipes."              │
│    },                                                                                                           │
│    {                                                                                                            │
│      "task_name": "As a user who frequently cooks for different group sizes, I want to be able to easily scale  │
│  recipes and shopping lists for different numbers of people, so I can avoid food waste and ensure I have        │
│  enough ingredients for everyone.",                                                                             │
│      "task_notes": "Develop a recipe scaling tool that allows users to adjust recipe quantities based on the    │
│  number of people they are cooking for. The tool should also automatically update the shopping list to reflect  │
│  the scaled quantities."                                                                                        │
│    },                                                                                                           │
│    {                                                                                                            │
│      "task_name": "As a busy parent, I want to quickly create a meal plan for the week that caters to my        │
│  family's dietary needs and preferences, so I can save time and reduce stress.",                                │
│      "task_notes": "Provide a streamlined workflow for creating weekly meal plans that takes into account       │
│  dietary needs and preferences. The goal is to make the meal planning process as quick and easy as possible     │
│  for busy parents."                                   